In [2]:
import numpy as np
import pandas as pd
import requests
import urllib.request
import os
import re
import json
import lxml
from pprint import pprint
from tqdm.notebook import tqdm as tqdm
from bs4 import BeautifulSoup

In [130]:
import numpy as np
import pandas as pd
import requests
import urllib.request
import os
import re
import json
import lxml
from pprint import pprint
from tqdm.notebook import tqdm as tqdm
from bs4 import BeautifulSoup


'''
:先通过Fiddler 4得到一部分需要的公众号历史文章数据，默认时间跨度从2020年11月底-2021年6月初，公众号文章数量较少的一直追溯到公众号第一篇文章；
:10个公众号列表：["机器之心","量子位","奇物论","芯东西","机器人大讲堂","高分子科学前沿","scienceAI","IT之家","DeepTech深科技","AI科技评论"]；
:处理Fiddler得到的json文件，得到公众号的历史文章数据

'''

def pre_process(str): 
    '''
    :处理json得到所需数据list
    '''
    global title, digest, url
    #用正则表达式提取标题和url，由于部分文章不存在副标题，为了表格长度一致性先不提取副标题，放在正文内容一起处理
    pattern_title = re.compile(r'(?<=title\":\").*?(?=\")', re.I)
    pattern_digest = re.compile(r'(?<=digest\":\").*?(?=\")', re.I)
    pattern_url = re.compile(r'(http://mp.weixin.qq.com/.*?wechat_redirect)', re.I)
    
    title = pattern_title.findall(str)
    digest = pattern_digest.findall(str)
    url = pattern_url.findall(str)

    for i in range(len(url) - 1, -1, -1):
        if url[i] == 'http://mp.weixin.qq.com/s?__biz=MzA4NDk3ODEwNQ==&amp;mid=203762979&amp;idx=1&amp;sn=71354892ae0a14b01e359accb376fcbe&amp;scene=25&amp;scene=25#wechat_redirect':
            url.remove(url[i])
        if len(title[i]) <= 3:
            title.remove(title[i])
            digest.remove(digest[i])
        if len(digest[i]) <= 3 and len(digest[i]) > 0:
            digest.remove(digest[i])

            
def list_to_dataframe(title, url):
    '''
    生成Dataframe
    '''

    global article_data
    article_data = {"title":title,
                    #"digest":digest,
                    "url":url}
    article_data = pd.DataFrame(article_data)
    return article_data

def add_time_and_text(whole_frame):
    '''
    :title和url处理完毕，进一步按总表url索引爬取time和text
    :增加两列，分别记录文章发布时间和正文部分的文字内容，基于获取的url来进一步获得(均为str)
    :其中time无法直接从网页元素获得，直接分析整个网页文本
    
    '''
    
    text=[]
    time=[]
    for i in tqdm(range(len(whole_frame['url']))):
        '''
        :循环得到一个text的list后再一并写入df，time同理
        '''
        
        try:                                           
            html = urllib.request.urlopen(whole_frame.iloc[i][1]).read()
        except http_client.IncompleteRead as e:
            html = e.partial
        #避免 IncompleteRead Error
        soup = BeautifulSoup(html)
        

        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        text1 = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text1.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text1 = '\n'.join(chunk for chunk in chunks if chunk)
        text.append(text1)  
        
        resp = requests.get(whole_frame.iloc[i][1])
        pattern_timestamp = re.compile(r'202[0-9]\-[0-1][0-9]\-[0-3][0-9]',re.I)
        timestamp = pattern_timestamp.findall(resp.text)
        try:
            time.append(timestamp[0])
        except:
            time.append(0)
        
    whole_frame['text']=text
    whole_frame['time']=time
        
        
#公众号列表：["机器之心","量子位","奇物论","芯东西","机器人大讲堂","高分子科学前沿","scienceAI","IT之家","DeepTech深科技","AI科技评论"]        
if __name__ == "__main__":
    '''
    :得到一个四列的df
    '''
    whole_frame = pd.DataFrame(columns=['title','url','time','text'])
    all_json=os.listdir(r'E:\WeChatFiles\rawlist\scienceAI')
    for j_name in all_json:
        # open json file
        j=open(r'E:\WeChatFiles\rawlist\scienceAI'+'\\'+j_name,'rb')

        # load info in json
        data=json.load(j)
        str = data['general_msg_list']
        pre_process(str)
        list_to_dataframe(title,url)

        whole_frame = whole_frame.append(article_data)
        whole_frame = whole_frame.fillna(0)
    add_time_and_text(whole_frame)   
    


##测试用
# j=open(r'E:\\WeChatFiles\\rawlist\\机器之心\\116_.json','rb')
# data=json.load(j)
# str = data['general_msg_list']
# pre_process(str)
# len(digest)"

  0%|          | 0/157 [00:00<?, ?it/s]

In [131]:
'''
:这部分用于删除异常文章，如已经由于违规被平台删除的文章，没有发布时间，通过检查time字段删除
:按时间升序排序后重置索引，生成csv文件
'''
whole_frame.reset_index(drop=True, inplace=True)
for i in range(len(whole_frame['time'])-1,-1,-1):
    if whole_frame.iloc[i][2]== 0 or whole_frame.iloc[i][2]=="":
        whole_frame.drop(labels = i,inplace=True)
whole_frame.sort_values("time",inplace=True)
whole_frame.reset_index(drop=True, inplace=True)
whole_frame.to_csv("scienceAI.csv",encoding='utf-8')

In [132]:
whole_frame.iloc[1][1]

'http://mp.weixin.qq.com/s?__biz=MzI3MjM3ODk0NQ==&amp;mid=2247483870&amp;idx=1&amp;sn=1a7152ebe72f5bb3b0965d63f06a3bf4&amp;chksm=eb323cf0dc45b5e6bb0b79c9b4e38a094299d5b0066f07040647f873b8fa3fa16686ea7a736c&amp;scene=27#wechat_redirect'

In [143]:
#公众号列表：["机器之心","量子位","奇物论","芯东西","机器人大讲堂","高分子科学前沿","scienceAI","IT之家","DeepTech深科技","AI科技评论"]
#用于sample的处理
# df = pd.read_csv('sample.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
# ss = df['text'].str.replace('\\\\n',' ').str.replace('微信扫一扫使用小程序',' ') \
#                     .str.replace('微信扫一扫关注该公众号',' ').str.replace('长按识别前往小程序',' ').str.replace('\\\\xa0',' ') \
#                     .str.replace('取消 允许 取消 允许 知道了  ',' ') \
#                     .str.replace('上一篇 下一篇 阅读 分享 收藏 赞 在看 已同步到看一看写下你的想法 前往“发现”-“看一看”浏览“朋友在看” 前往看一看 看一看入口已关闭 在“设置”-“通用”-“发现页管理”打开“看一看”入口 我知道了 已发送 取消   发送',' ').str.replace('      ','')
# df['text'] = ss
# df.to_csv('sample.csv',encoding='utf-8')
# for i in range(len(df['time'])):
#     print(df['time'][i])

<ipython-input-143-763dbeff544e>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  ss = df['text'].str.replace('\\\\n',' ').str.replace('微信扫一扫使用小程序',' ') \


In [147]:
df = pd.read_csv('sample.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
df

,title,url,time,text
0,世界首个「双边植入」脑机接口！瘫痪人士用「意念」操控双机械臂吃蛋糕,https://mp.weixin.qq.com/s?__biz=MzI5MzE0NDUzN...,2021/1/5,世界首个「双边植入」脑机接口！瘫痪人士用「意念」操控双机械臂吃蛋糕 原创 Robospeak...
1,Nature连发两篇光子AI芯片论文！光子计算时代已至？,https://mp.weixin.qq.com/s?__biz=MjM5NTM4MTUyM...,2021/1/7,Nature连发两篇光子AI芯片论文！光子计算时代已至？ 原创 心缘 芯东西 芯东西 微信号...
2,燕山大学「无系留软体抓手」，1.5kg自重抓起40kg物体，还能陪你打羽毛球！,https://mp.weixin.qq.com/s?__biz=MzI5MzE0NDUzN...,2021/1/13,燕山大学「无系留软体抓手」，1.5kg自重抓起40kg物体，还能陪你打羽毛球！ 原创 卡卡罗...
3,让机器鱼混进鱼群内部，哈佛科学家研发出鱼群机器人,https://www.ithome.com/0/530/774.htm,2021/1/18,让机器鱼混进鱼群内部，哈佛科学家研发出鱼群机器人 - IT之家 首页 IT圈 辣品 设置 日...
4,引燃AI社区，不用跨界也能从文本生成图像，OpenAI新模型打破自然语言与视觉次元壁,https://mp.weixin.qq.com/s?__biz=MzA3MzI4MjgzM...,2021/1/6,引燃AI社区，不用跨界也能从文本生成图像，OpenAI新模型打破自然语言与视觉次元壁 机器之...
5,给AI从业者的重要年终参考——谷歌两万字博客总结自身2020人工智能进展,https://mp.weixin.qq.com/s?__biz=MzI3MjM3ODk0N...,2021/1/17,给AI从业者的重要年终参考——谷歌两万字博客总结自身2020人工智能进展 ScienceAI...
6,不再局限于视觉！MIT让机器人通过触摸即可与周围环境进行交互,https://c.m.163.com/news/a/G120GNMB05119734.ht...,2021/1/23,不再局限于视觉！MIT让机器人通过触摸即可与周围环境进行交互subtitleDeepTech...
7,腾讯 Robotics X 实验室?发布首个全自研四足机器人Max，有腿又有轮，会「拜年讨红包」,https://mp.weixin.qq.com/s?__biz=MzkwNDIwOTAwO...,2021/3/2,腾讯 Robotics X 实验室\u200b发布首个全自研四足机器人Max，有腿又有轮，会...
8,一种电子皮肤，两篇Nature Materials,https://mp.weixin.qq.com/s?__biz=Mzg5ODE3NzY2M...,2021/2/22,一种电子皮肤，两篇Nature Materials 原创 小奇 奇物论 奇物论 微信号 Ma...
9,深度学习后，图灵奖得主Bengio研究核心是什么？因果表示学习,https://mp.weixin.qq.com/s?__biz=MzA3MzI4MjgzM...,2021/2/25,深度学习后，图灵奖得主Bengio研究核心是什么？因果表示学习 机器之心 机器之心 微信号 ...


In [130]:
'''
This part is going to download the webpage
'''

def SaveFile(fpath,fileContent):
    with open(fpath, 'w', encoding='utf-8') as f:
            f.write(fileContent)
#定义一个下载url网页并保存的方法
def DownLoadHtml(url):
    #构造请求头    
    headers = {
                 'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
                 'Accept':'*/*', 
                 'Connection':'keep-alive', 
                 'Accept-Language':'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7'
              }     
#模拟浏览器发送请求     
    response = requests.get(url,headers = headers)    
    if response.status_code == 200:#返回码为200表示正常返回
        htmltxt = response.text #网页正文  
        return htmltxt     
    else:        
        return None 
    
if __name__ == "__main__": 
    for i in tqdm(range(len(whole_frame['url']))):
        url = whole_frame.iloc[i][1]
        htmlstr = DownLoadHtml(url)
        savepath="E:/WeChatFiles/scienceAI/"+whole_frame.iloc[i][0].replace('|','').replace('/','').replace('?','').replace(':','').replace('\\','').replace('<','').replace('>','').replace('"','')+".html" #先手动在C盘下新建vWeChatFiles文件夹       
        SaveFile(savepath,htmlstr)

NameError: name 'whole_frame' is not defined

In [156]:
try:                                           
    html = urllib.request.urlopen('https://mp.weixin.qq.com/s?__biz=MzI5MzE0NDUzNQ==&mid=2650241733&idx=1&sn=f2cac844506a0b2773ffa51cd728e61b&scene=21#wechat_redirect').read()
except http_client.IncompleteRead as e:
    html = e.partial
#避免 IncompleteRead Error
soup = BeautifulSoup(html)
        

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text1 = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text1.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text1 = '\n'.join(chunk for chunk in chunks if chunk)
text1

'Nature 子刊：史上首个功能齐全的形态自适应机器人，可根据地形自动变化形态\n机器人大讲堂\n机器人大讲堂\n微信号\nRoboSpeak\n功能介绍\n机器人大讲堂是立德共创服务平台旗下引领行业发展的新媒体平台，已举办多种形式系列活动近百场，行业媒体矩阵垂直粉丝30万余人；立德研究院承接智库咨询和科研项目；立德孵化基于顶尖专家优势形成早期高技术成果产业化。\n收录于话题\n近日，来自奥斯陆大学（University of Oslo）信息学院的研究团队成功研制出了一种新型四足机器人，它不仅能够感知地形变换，同时还能根据地形变换自动改变形态。这也是人类历史上第一个功能齐全的形态自适应机器人。相关论文以 “Real-world embodied AI through a morphologically adaptive quadruped robot” 为题，于 本月发表在科学期刊 Nature 子刊《自然-机器智能》（Nature Machine Intelligence）上。\u200d\u200d这个四足机器人被称为Dyret，在非结构化的户外环境中，它能自动适应不同的条件。四足机器人Dyret可以调整腿长，使身体适应地表。在行走中，它学会了最适用的行走方式。这样，下次遇到未知环境时，它会表现得更好。Dyret（挪威语中“动物”的意思）是动态测试机器人的首字母缩写。“我们已经展示了让机器人不断调整身体形状的好处。我们的物理机器人也证明了，用当今的技术很容易做到这一点。”奥斯陆大学信息学系的高级讲师Tnnes Nygaard说。在Dyret的例子中，改变身体形状意味着它可以调整腿的长度。副教授Kyre Glette说：“我们已经看到，一种调整身体形状的机制对我们的机器人很有用，我们相信这也可以应用于其他机器人设计。”此前，他们已经证明，这种机器人能够在室内受控条件下适应不同的环境。然后，澳大利亚英联邦科学和工业研究组织的其他机器人研究员一起工作了半年，专门在户外测试自学机器人。“这以前被认为在现实世界中很难实现。通过机器人和我们的实验，我们已经证明了这是可能的。”Nygaard介绍说。通过改变腿的长度，它可以自动转换成不同的体型。形态自适应机器人可以在不可预知的环境中工作，解决新的任务，而无需每次面对意料之外的事情都进行重新设计或重建。▍机器人的一大步对我们人

In [5]:
#公众号列表：["机器之心","量子位","奇物论","芯东西","机器人大讲堂","高分子科学前沿","scienceAI","IT之家","DeepTech深科技","AI科技评论"]
df1 = pd.read_csv('机器之心.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
df2 = pd.read_csv('量子位.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
df3 = pd.read_csv('奇物论.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
df4 = pd.read_csv('芯东西.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
df5 = pd.read_csv('机器人大讲堂.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
df6 = pd.read_csv('高分子科学前沿.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
df7 = pd.read_csv('scienceAI.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
df8 = pd.read_csv('IT之家.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
df9 = pd.read_csv('DeepTech深科技.csv',index_col=0,encoding='utf-8',error_bad_lines = False)
df10= pd.read_csv('AI科技评论.csv',index_col=0,encoding='utf-8',error_bad_lines = False)

In [79]:
#合并各个公众号的表格
df_total = df1.append(df2).append(df3).append(df4).append(df5).append(df6).append(df7).append(df8).append(df9).append(df10)
df_total.sort_values("time",inplace=True)
df_total.reset_index(drop=True, inplace=True)

In [81]:
s = df_total['text'].str.replace('\n',' ').str.replace('微信扫一扫使用小程序',' ') \
                    .str.replace('微信扫一扫关注该公众号',' ').str.replace('长按识别前往小程序',' ').str.replace('发送到看一看',' ') \
                    .str.replace('取消 允许 取消 允许 知道了  ',' ') \
                    .str.replace('上一篇 下一篇 阅读原文 阅读 分享 收藏 赞 在看 已同步到看一看写下你的想法 前往“发现”-“看一看”浏览“朋友在看” 前往看一看 看一看入口已关闭 在“设置”-“通用”-“发现页管理”打开“看一看”入口 我知道了 已发送 取消   发送',' ')
df_total['text'] = s
df_total.to_csv('total_hist_article.csv',encoding = 'utf-8')

In [149]:
df_total

,title,url,time,text
0,阿里定向广告新一代主模型：基于搜索的超长用户行为建模范式,http://mp.weixin.qq.com/s?__biz=MzA3MzI4MjgzMw...,2020-01-07,阿里定向广告新一代主模型：基于搜索的超长用户行为建模范式 机器之心 机器之心 微信号 alm...
1,投资者说丨“辉煌的2020年”，中美医疗领域IPO均破往年纪录，背后有哪些细节与风险？,http://mp.weixin.qq.com/s?__biz=MzA3NTIyODUzNA...,2020-02-01,投资者说丨“辉煌的2020年”，中美医疗领域IPO均破往年纪录，背后有哪些细节与风险？ De...
2,清华姚班校友马腾宇斩获斯隆奖！与鬲融师承一脉，李飞飞点赞祝贺,http://mp.weixin.qq.com/s?__biz=MzIzNjc1NzUzMw...,2020-02-23,清华姚班校友马腾宇斩获斯隆奖！与鬲融师承一脉，李飞飞点赞祝贺 原创 关注前沿科技 量子位 量...
3,全球首个 AI 说唱歌手 TikTok 发新歌，虚拟偶像正当时,http://mp.weixin.qq.com/s?__biz=MzA5ODEzMjIyMA...,2020-03-06,全球首个 AI 说唱歌手 TikTok 发新歌，虚拟偶像正当时 AI科技评论 AI科技评论 ...
4,粗犷营销时代结束，获客愈发困难，以下24个线上智能获客方案让企业足不出户把握商机,http://mp.weixin.qq.com/s?__biz=MzA3MzI4MjgzMw...,2020-03-17,粗犷营销时代结束，获客愈发困难，以下24个线上智能获客方案让企业足不出户把握商机 原创 机器...
...,...,...,...,...
10091,不用GPU，稀疏化也能加速你的YOLOv3深度学习模型,http://mp.weixin.qq.com/s?__biz=MzIzNjc1NzUzMw...,2021-06-02,不用GPU，稀疏化也能加速你的YOLOv3深度学习模型 关注前沿科技 量子位 量子位 微信号...
10092,机器人会模仿人类微笑了，但我总觉得这笑容……,http://mp.weixin.qq.com/s?__biz=MzIzNjc1NzUzMw...,2021-06-02,机器人会模仿人类微笑了，但我总觉得这笑容…… 关注前沿科技 量子位 量子位 微信号 Qbit...
10093,英伟达3080Ti、3070Ti来了：继续封锁挖矿性能，网友：不信，空气卡+1,http://mp.weixin.qq.com/s?__biz=MzIzNjc1NzUzMw...,2021-06-02,英伟达3080Ti、3070Ti来了：继续封锁挖矿性能，网友：不信，空气卡+1 关注前沿科技...
10094,每周工作超55小时会增加死亡风险！科研人员还好吗？,http://mp.weixin.qq.com/s?__biz=MzA5NjM5NzA5OA...,2021-06-02,每周工作超55小时会增加死亡风险！科研人员还好吗？ 高分子科学前沿 高分子科学前沿 微信号 ...


In [148]:
df_total['text'][10094]

'每周工作超55小时会增加死亡风险！科研人员还好吗？ 高分子科学前沿 高分子科学前沿 微信号 Polymer-science 功能介绍 「高分子科学前沿」关注高分子、材料、化学领域全球最新科研进展！坚持“有趣、有料、前沿”，入选“2018年度十大明星学术公众号”。关注我，为高分子行业点赞助力！投稿、荐稿、合作请联系邮箱：editor@polysci.cn 收录于话题 以下文章来源于中国生物技术网 ，作者生物技术君 中国生物技术网 中国生物技术信息网官方微信平台 过劳，即长时间过度工作（加班）。世界上，首宗记录过劳死的案例发生在日本。因此，“过劳死”一词也是日本人发明的。日本也一直被认为是过劳大国。然而，过劳现象如同流行病一样已在全球蔓延。在中国，互联网大厂以及许多企业的“996”工作制已成常态，上班需要准时打卡，而下班时间却成为永远的变量。众所周知，这种工作模式肯定会对身体健康造成负面影响。北京时间5月18日，发表在《Environment International》上的一项新研究中，来自世界卫生组织（WHO）、国际劳工组织（ILO）和美国加州大学旧金山分校等国际研究机构发表了世界范围内首份关于过劳对健康影响的报告。他们分析了全球近200个国家或地区的相关数据，发现过劳导致全球每年74.5万人死亡。关键是，这一问题正变得越来越严重。该研究提出，保护和促进职业和职工的安全和健康需要干预措施，以减少因工作时间过长带来的健康风险。在这项研究中，世卫组织和劳工组织等机构的研究人员首次对长时间工作造成的生命和健康损失进行了全球分析。他们系统回顾了几十项调查缺血性心脏病和中风的荟萃研究，分析了涵盖194个不同国家或地区的数据。虽然统计数据没有显示出明确的因果关系，但根据世界卫生组织的标准，它们确实提供了工作时间过长有害健康的足够证据。与标准工作时间（每周35-40小时）相比，每周工作55小时或更长的人患缺血性心脏病和中风的风险更高。该研究显示，在2016年，全球有4.88亿人（约占总人口的8.9%）每周至少工作55小时或更长。而且这一比例正在快速增加，从2000年到2016年，每周至少工作55小时的人口比例增加了9％。长时间工作造成的疾病负担约占总疾病负担的三分之一，因此被确定为职业病负担最大的危险因素。2016年，工作时间过长（55小时及以上）导致约74.5万人死于中风和缺